# Springboard Data Science Capstone 2
Identifying Credit Card Fraud: Data Wrangling

Mark Cohen

2023-3-7

## 1. Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../src")
import data_utils

In [16]:
# Confirming that data is present locally and downloading and unzipping if not.
data_utils.raw_data_on_disk()

Data already present. Skipping download.


## 2. Inspecting the data files

### 2.1 A sample record

The data set includes the first transaction record separated out into its own csv file.

The main transactions data file includes 20 million records. This is too large a scale for the limited computing resources available for this project.